In [5]:
PATH = "/Users/joregan/Playing/hu-tts/hungarian-single-speaker-tts"

In [ ]:
!pip install datasets librosa

In [ ]:
from pathlib import Path
import datasets

In [6]:
path = Path(PATH)
outdir = path / "data"
transcript = path / "transcript.txt"

In [40]:
def clean_text(text, norm = False):
    if text.endswith(" -"):
        text = text[:-2]
        text = text.replace(".S ", ". S ")
    if norm:
        text = text.replace(". ", " ")
        if len(text) > 1 and text[-1] in [".", "!", "?", ":"]:
            text = text[:-1]
        text = text.replace(" -, ", " ")
        text = text.replace(" - ", " ")
        text = text.replace(". ", " ")
        text = text.replace(", ", " ")
        text = text.replace("? ", " ")
        text = text.replace(": ", " ")
        text = text.replace("! ", " ")
        text = text.lower()
    return text

In [46]:
data = []
with open(str(transcript)) as ts:
    for line in ts.readlines():
        data.append(line.strip().split("|"))

In [49]:
def data_gen():
    for i in data:
        filepath = i[0]
        fileid = filepath.split("/")[1].replace(".wav", "")
        text = i[1]
        lightly_cleaned = clean_text(text)
        fully_cleaned = clean_text(text, True)
        yield {
            "id": fileid,
            "audio": str(path / filepath),
            "original_text": lightly_cleaned,
            "text": fully_cleaned,
            "duration": float(i[3])
        }

In [ ]:
from datasets import Dataset, Audio
dataset = Dataset.from_generator(data_gen).cast_column("audio", Audio())

In [52]:
dataset.to_parquet(outdir)

1715550

In [ ]:
dataset[0]